# Ejercicios Pair Programming - Modulo 3 - Sprint 1 
## Lección 8 - Encoding
### Pilar García y Sonia Ruiz Pérez

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

Vuestro set de datos debería tener al menos una variable categórica, el objetivo del pair programming de hoy:

- Hacer una codificación de la/las variables categóricas que tengáis en vuestro set de datos.
- Recordad que lo primero que deberéis hacer es decidir su vuestras variables tienen o no orden, para que en función de esto uséis una aproximación u otra.
- Guardad el dataframe, donde deberíais tener las variables estadandarizas, normalizadas y codificadas en un csv para usarlo en el próximo pairprogramming.

Happy coding 🤯📊

In [75]:
import pandas as pd
pd.options.display.max_columns=None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [76]:
df = pd.read_csv("files/taxis3_estandar.csv", index_col=0)
df.head(3)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough,tip_no_zeros,tip_box,passengers_robust,distance_robust,fare_robust,tolls_robust,total_robust
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan,2.150000e+00,0.787753,0.0,-0.022624,-0.294118,0.0,-0.127368
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan,1.000000e-10,-10.996744,0.0,-0.389140,-0.529412,0.0,-0.511579
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan,2.360000e+00,0.886769,0.0,-0.126697,-0.235294,0.0,0.000000


`En nuestro dataframe tenemos las columnas categóricas color, payment, pickup_borough y dropoff_borough.`
` Que son a las que le vamos a realizar un encoding.`
` La vamos a dividir en dos grupos, las que tienen y las que no tienen orden en sus categorías.`

### `Variables que no tienen orden.`
#### `Color`
`Color es una variable con dos categorías, 'yellow' y 'green'`
#### `Método a aplicar: One Hot Encoder`

In [77]:
def aplicar_one_hot_encoder(df, columna):
    # Iniciar el método
    ohe = OneHotEncoder(categories="auto", drop=None, dtype=np.int64)

    # Hacer la codificación de los datos para la variable a trasformar
    transformados = ohe.fit_transform(df[[columna]])

    # Convertir el array obtenido en un dataframe
    ohe_df = pd.DataFrame(transformados.toarray())

    # Obtener el nombre de las nuevas columnas
    ohe_df.columns = ohe.get_feature_names()

    # Concatenar el dataframe original con el dataframe obtenido
    df_unido = pd.concat([df, ohe_df], axis=1)

    # Eliminar columna original
    df_unido.drop(columna, axis=1, inplace=True)

    return df_unido

In [78]:
# Aplicamos la función
df = aplicar_one_hot_encoder(df, "color")

In [79]:
df.head(3)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough,tip_no_zeros,tip_box,passengers_robust,distance_robust,fare_robust,tolls_robust,total_robust,x0_green,x0_yellow
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan,2.150000e+00,0.787753,0.0,-0.022624,-0.294118,0.0,-0.127368,0,1
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan,1.000000e-10,-10.996744,0.0,-0.389140,-0.529412,0.0,-0.511579,0,1
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,credit card,Alphabet City,West Village,Manhattan,Manhattan,2.360000e+00,0.886769,0.0,-0.126697,-0.235294,0.0,0.000000,0,1


`Este método nos elimina la columna original color y ahora nos aparecen dos nuevas columnas, una por cada categorías llamadas x0_green y x0_yellow. El valor que toma cada celda es 1 si cumple la categoría y 0 si no la cumple.`

### `Variables que tienen orden.`
#### `pickup_borough`
`pickup_borough es una variable con cuatro categorías, 'Manhattan', 'Queens', 'Brooklyn' y 'Bronx'`
` Tras realizar el test de ANOVA conocemos la fuerza de cada una de las categorías, siendo Queens la categoría con mayor efecto, seguido de Manhattan, Brooklyn y Bronx, por lo que vamos a darle un valor de 3 a la categoría con más efecto y 0 a la que menos.`
#### `Método a aplicar: Ordinal Encoding`

In [80]:
def aplicar_ordinal_encoder(df, columna, orden_valores):
    # Iniciar el método
    oe = OrdinalEncoder(categories=[orden_valores], dtype=int)

    # Hacer la codificación de los datos para la variable a trasformar
    transformados_oe = oe.fit_transform(df[[columna]])

    # Convertir el array obtenido en un dataframe
    oe_df = pd.DataFrame(transformados_oe)

    # Cambiar el nombre de las nuevas columnas
    #oe_df.columns = oe.feature_names_in_

    columna += "_oe"

    # Sobreescribir la columna
    df[columna] = oe_df

    return df

In [81]:
orden = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens']

In [82]:
df = aplicar_ordinal_encoder(df, "pickup_borough", orden)

In [86]:
df.sample(5)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough,tip_no_zeros,tip_box,passengers_robust,distance_robust,fare_robust,tolls_robust,total_robust,x0_green,x0_yellow,pickup_borough_oe
2950,2019-03-22 00:04:23,2019-03-22 00:22:33,1,5.82,19.50,0.00,0.0,23.30,credit card,Upper East Side South,Washington Heights South,Manhattan,Manhattan,1.000000e-10,-10.996744,0.0,1.886878,1.176471,0.0,0.962105,0,1,2
4106,2019-03-28 07:33:20,2019-03-28 08:15:51,1,9.16,31.67,0.00,0.0,32.47,credit card,West Concourse,East Chelsea,Bronx,Manhattan,1.000000e-10,-10.996744,0.0,3.398190,2.608235,0.0,1.927368,0,1,0
1368,2019-03-01 19:26:40,2019-03-01 19:45:38,1,3.00,14.50,3.75,0.0,22.55,credit card,Penn Station/Madison Sq West,Lenox Hill East,Manhattan,Manhattan,3.750000e+00,1.389139,0.0,0.610860,0.588235,0.0,0.883158,0,1,2
5669,2019-03-04 23:29:19,2019-03-04 23:34:26,1,0.97,6.00,1.82,0.0,9.12,credit card,Brooklyn Heights,Columbia Street,Brooklyn,Brooklyn,1.820000e+00,0.612418,0.0,-0.307692,-0.411765,0.0,-0.530526,1,0,1
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.00,0.00,0.0,9.30,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan,1.000000e-10,-10.996744,0.0,-0.389140,-0.529412,0.0,-0.511579,0,1,2


`Vemos que nos ha creado correctamente la nueva con el orden de valores especificados.`

### `Variables que tienen orden.`
#### `dropoff_borough`
`dropoff_borough es una variable con cinco categorías, 'Manhattan', 'Queens', 'Brooklyn', 'Bronx' y 'Staten Island'`
` Tras realizar el test de ANOVA conocemos la fuerza de cada una de las categorías, siendo solo 'Staten Island' la categoría con  efecto, por lo que vamos a darle un valor de 1 a esta categoría y 0 al resto.`
#### `Método a aplicar: map`

In [99]:
def aplicar_map(df, columna, diccionario):

    columna_nueva = columna + "_map"

    # Aplicar map
    df[columna_nueva] = df[columna].map(diccionario)

    return df

In [100]:
# Creamos el diccionario con los valores únicos 
mapa = {'Manhattan': 0, 'Queens': 0, 'Brooklyn': 0, 'Bronx': 0, 'Staten Island': 1}

In [101]:
# aplicamos la función
df = aplicar_map(df, "dropoff_borough", mapa)

In [97]:
df.sample(5)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough,tip_no_zeros,tip_box,passengers_robust,distance_robust,fare_robust,tolls_robust,total_robust,x0_green,x0_yellow,pickup_borough_oe,dropoff_boroughmap
3042,2019-03-25 16:18:47,2019-03-25 16:45:18,1,8.54,24.74,0.00,0.0,25.54,credit card,Kips Bay,Hamilton Heights,Manhattan,Manhattan,1.000000e-10,-10.996744,0.0,3.117647,1.792941,0.0,1.197895,0,1,2,0
376,2019-03-24 08:53:31,2019-03-24 08:57:29,3,1.03,5.50,1.76,0.0,10.56,credit card,Chinatown,Greenwich Village South,Manhattan,Manhattan,1.760000e+00,0.577408,2.0,-0.280543,-0.470588,0.0,-0.378947,0,1,2,0
4780,2019-03-07 23:11:42,2019-03-07 23:28:15,1,7.16,22.00,1.00,0.0,26.80,credit card,Clinton East,Washington Heights South,Manhattan,Manhattan,1.000000e+00,0.000000,0.0,2.493213,1.470588,0.0,1.330526,0,1,2,0
3625,2019-03-22 19:21:22,2019-03-22 19:29:00,5,1.34,7.00,0.00,0.0,11.30,cash,Times Sq/Theatre District,Central Park,Manhattan,Manhattan,1.000000e-10,-10.996744,4.0,-0.140271,-0.294118,0.0,-0.301053,0,1,2,0
6070,2019-03-08 16:45:03,2019-03-08 16:53:28,1,1.32,8.00,1.96,0.0,11.76,credit card,Jackson Heights,Astoria,Queens,Queens,1.960000e+00,0.690128,0.0,-0.149321,-0.176471,0.0,-0.252632,1,0,3,0


In [102]:
df["dropoff_borough"].value_counts()

Manhattan        5172
Queens            534
Brooklyn          497
Bronx             136
Staten Island       2
Name: dropoff_borough, dtype: int64

In [103]:
df["dropoff_borough_map"].value_counts()

0    6339
1       2
Name: dropoff_borough_map, dtype: int64